In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, mean_squared_error, accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import SGD

In [4]:
df = pd.read_csv('player_df.csv')
df = df.drop(columns='player')
df.head()

,score_team,score_opp,map_pick,rating,acs,kill,death,assist,kast%,adr,...,map_Fracture,map_Haven,map_Icebox,map_Lotus,map_Pearl,map_Split,agent_class_controller,agent_class_duelist,agent_class_initiator,agent_class_sentinel
0,5,13,False,1.19,239.0,15,11,6,72.0,146.0,...,False,False,False,False,False,False,False,False,True,False
1,5,13,False,0.84,234.0,15,14,6,61.0,145.0,...,False,False,False,False,False,False,False,False,False,True
2,5,13,False,0.68,173.0,10,15,2,67.0,123.0,...,False,False,False,False,False,False,False,True,False,False
3,5,13,False,0.63,142.0,8,15,4,83.0,92.0,...,False,False,False,False,False,False,False,False,True,False
4,5,13,False,0.57,114.0,7,16,8,61.0,63.0,...,False,False,False,False,False,False,True,False,False,False


In [5]:

train, test = train_test_split(df, train_size=0.75)
print(f'Train size: {train.shape} \nTest size: {test.shape}')

Train size: (4672, 26) 
Test size: (1558, 26)


In [6]:
x_train = train.drop(columns=['team_win','score_team','score_opp'])
x_test = test.drop(columns=['team_win','score_team','score_opp'])
y_train = train['team_win']
y_test = test['team_win']

## Simple Naive Bayes Classification

In [7]:
NB_model = GaussianNB()
NB_model.fit(x_train, y_train)
y_pred = NB_model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.72      0.74      0.73       791
        True       0.72      0.70      0.71       767

    accuracy                           0.72      1558
   macro avg       0.72      0.72      0.72      1558
weighted avg       0.72      0.72      0.72      1558



In [8]:
print(f'Overall accuracy: {accuracy_score(y_pred,y_test)}')

Overall accuracy: 0.7188703465982028


## Support Vector Classifier

In [9]:
# Linear SVC
lin_SVC = SVC(kernel='linear')
lin_SVC.fit(x_train, y_train)
y_pred = lin_SVC.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.78      0.80      0.79       791
        True       0.79      0.76      0.78       767

    accuracy                           0.78      1558
   macro avg       0.78      0.78      0.78      1558
weighted avg       0.78      0.78      0.78      1558



In [10]:
# rbf kernel
rbf_SVC = SVC(kernel='rbf')
rbf_SVC.fit(x_train, y_train)
y_pred = rbf_SVC.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.73      0.75      0.74       791
        True       0.73      0.71      0.72       767

    accuracy                           0.73      1558
   macro avg       0.73      0.73      0.73      1558
weighted avg       0.73      0.73      0.73      1558



## Artificial Neural Network

In [11]:
nn_model = Sequential()
nn_model.add(Dense(units = 12, input_dim = x_train.shape[1], activation = 'softmax'))
nn_model.add(Dense(units = 24, activation = 'relu'))
nn_model.add(Dense(units = 15, activation = 'relu'))
nn_model.add(Dense(units = 1, activation='sigmoid'))

c:\Users\Chris Mo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
opt = SGD(learning_rate=0.02)
nn_model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
nn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 24)             │           312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           375 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 991 (3.87 KB)

 Trainable params: 991 (3.87 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
nn_model.fit(x_train,y_train, epochs=100)

Epoch 1/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.4992 - loss: 0.6954
Epoch 2/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - accuracy: 0.4950 - loss: 0.6933
Epoch 3/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - accuracy: 0.5046 - loss: 0.6934
Epoch 4/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.5015 - loss: 0.6933
Epoch 5/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - accuracy: 0.4827 - loss: 0.6936
Epoch 6/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - accuracy: 0.5039 - loss: 0.6933
Epoch 7/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.5042 - loss: 0.6933
Epoch 8/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.4970 - loss: 0.6935
Epoch 9/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.4670 - loss: 0.6936
Epoch 10/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.4965 - loss: 0.6935
Epoch 11/100
146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.4970 - loss: 0.6934
Epoch 12/100
146/14

In [14]:
y_pred = nn_model.predict(x_test)
y_pred = [1 if y >= 0.5 else 0 for y in y_pred]

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [15]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.51      0.98      0.67       791
        True       0.37      0.01      0.02       767

    accuracy                           0.50      1558
   macro avg       0.44      0.50      0.34      1558
weighted avg       0.44      0.50      0.35      1558



## Hyperparameter Tuning